# Web Scraping MUTHead with BeautifulSoup and Requests 
## + Plotly
- ** Matthew Johnson, Oct 6, 2018**

In [1]:
%matplotlib inline
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt 
import seaborn as sns
import urllib
import pandas as pd
import requests
import warnings; warnings.simplefilter('ignore')  
import time
import re

**Method to collect stats given a BeautifulSoup item:**

In [109]:
prefix = 'https://www.muthead.com'

valid_traits = ['Clutch', 'Covers Ball', 'Fights for Extra Yards', 'Makes Aggressive Catches', 'Makes RAC Catches', 
                'Makes Possession Catches', 'Drops Open Passes', 'Makes Sideline Catches', 'High Motor', 'Penalty']

class Player:
    
    def __init__(self, link):
        self.stat_dict = {}
        self.trait_dict = {}
        self.name = ' '.join(link.split('-')[1:])
        self.link = link
        self.full_link = prefix + link #+ f'/upgrades?from={upgrade_number}'
        self.price = '0K'
        self.price_diff = '0%'
        self.traits_correct = 0
        self.ovr = 0
        self.position = '-'
        self.program = '-'
        self.first_name = ''
        self.last_name = ''
        self.team = '-'
        self.height_wt = ''
        self.cap_hit = 0
        
    def get_ratings(self):
        response = requests.get(self.full_link)
        #print(self.full_link)
        soup = BeautifulSoup(response.text, 'html.parser')
        self.soup = soup
        tag_list = []
        stat_list = []
        
        self.get_player_ratings()
        self.get_player_traits()
        #print(stat_list)
        #self.stat_list = stat_l
        #player_ratings_small = [self.read_rating_string(stat.string) for stat in stat_list]
        #print(player_ratings_small)
        #player_traits = [self.get_player_trait(trait) for trait in tag_list[200:300]]
        #print(player_traits)
        
        if len(self.trait_dict.keys()) == 10: self.traits_correct = 1
        self.get_price(self.soup)
        self.set_values(self.soup)
        
    def summary(self):
        print(self.name, ' - ', self.price, ' - ', self.traits_correct)
        
    def get_price(self, soup):
        try:
            self.price = soup.find_all('span', class_='item-price')[1].contents[0]
            self.price_diff = soup.find_all('span', class_='item-price')[1].contents[1]
        except:
            pass
        
    def get_player_ratings(self):
        stat_vals = self.soup.find_all('span', class_='stat') # name also stat-abbreviation
        stat_abbrevs = self.soup.find_all('span', class_='stat-abbreviation')
        [self.stat_dict.update({stat_abbrev.string : stat_val.string}) 
             for stat_abbrev, stat_val in zip(stat_abbrevs, stat_vals)];
    
    # 209-218?
    def get_player_traits(self):
        traits = self.soup.find('ul', class_='player-traits-list').contents

        for trait in traits:
            if trait != '\n': 
                try:
                    trait_key = str(trait).split('>')[2].rstrip('</span')
                    trait_val = str(trait).split('>')[3].rstrip('</li').strip()
                    self.trait_dict.update({trait_key: trait_val})
                except:
                    pass
        
    def set_values(self, soup):
        try: self.ovr = int(soup.find('span', class_='overall').string)
        except: pass
        try: self.pos = soup.find('span', class_='position').string
        except: pass
        try: self.program = soup.find('div', class_='program-name').string
        except: pass
        try: self.first_name = soup.find('span', class_='first-name').string
        except: pass
        try: self.last_name = soup.find('span', class_='last-name').string
        except: pass
        try: self.team = soup.find('span', class_='team').string
        except: pass
        try: self.height_wt = soup.find('span', class_='height-weight').string
        except: pass
        try: self.cap_hit = int(soup.find('span', class_='cap-hit').string)
        except: pass
        
    def get_stats_df(self):
        return pd.DataFrame({self.name : self.stat_dict}).T
    
    def get_traits_df(self):
        return pd.DataFrame({self.name : self.trait_dict}).T

**Getting player links from the search page filtered with WRs and set to 88+ OVR:**

In [10]:
# position = 8 <- WR


class PlayerHandler:
    
    def __init__(self):
        self.player_list = []
        self.max = 5
        self.MIN_OVR = 90
        self.player_links = []
    
    def get_player_links(self):
        links = []

        i=0
        while(len(links) < self.max):
            try:
                url = f'https://www.muthead.com/19/players?filter-market=3&filter-ovr-min=90&filter-position=8&page={i}'
                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'lxml')
    
                for link in soup.findAll('a', attrs={'href': re.compile("19/players/")}):
                    links.append(link.get('href'))
                i+=1
                
            except:
                print("## error")
                break
        
        self.player_links = [s for s in links if 'prices' not in s]


    def make_player_list(self):

        for player_link in self.player_links:
            p = Player(player_link)
            self.player_list.append(p)
        
        for player in self.player_list:
            player.get_ratings()
            time.sleep(1)
            
    def handle_players(self):
        self.get_player_links()
        self.make_player_list()
        return self.player_list
        


In [20]:
links = []
max_players = 5

i=0
while(len(links) < max_players):
            try:
                url = f'https://www.muthead.com/19/players?filter-market=3&filter-ovr-min=90&filter-position=8&page={i}'
                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'lxml')
    
                for link in soup.findAll('a', attrs={'href': re.compile("19/players/")}):
                    links.append(link.get('href'))
                i+=1
                
            except:
                print("## error")
                break
        
player_links = [s for s in links if 'prices' not in s]

In [110]:
player_list = []

for player_link in player_links[:1]:
            p = Player(player_link)
            player_list.append(p)
            


In [111]:
for player in player_list:
            player.get_ratings()
            time.sleep(1)

In [112]:
player1 = player_list[0]
player1.program

'MUT Monsters'

In [113]:
player1.trait_dict

{'Clutch': 'Yes',
 'Covers Ball': 'Brace for All Hits',
 'Drops Open Passe': 'No',
 'Fights for Extra Yard': 'Yes',
 'High Motor': 'Yes',
 'Makes Aggressive Catche': 'Yes',
 'Makes Possession Catche': 'Yes',
 'Makes RAC Catche': 'Yes',
 'Makes Sideline Catche': 'Yes',
 'Penalty': 'Disciplined'}

In [97]:
 # 3, 5, 7, ... 

a = player1.soup.find('ul', class_='player-traits-list').contents
a

for _ in a:
    if _ != '\n': 
        try:
            trait_key = str(_).split('>')[2].rstrip('</span')
            trait_val = str(_).split('>')[3].rstrip('</li').strip()
            print(trait_key, ':', trait_val)
        except:
            pass

Clutch : Yes
Covers Ball : Brace for All Hits
Fights for Extra Yard : Yes
Makes Aggressive Catche : Yes
Makes RAC Catche : Yes
Makes Possession Catche : Yes
Drops Open Passe : No
Makes Sideline Catche : Yes
High Motor : Yes
Penalty : Disciplined


In [75]:
temp = str(player1.soup.find('ul', class_='player-traits-list').contents[1])
trait_key = str(temp).split('>')[2].rstrip('</span')
trait_val = str(temp).split('>')[3].rstrip('</li').strip()

'Yes'

In [86]:
temp = player1.soup.find('ul', class_='player-traits-list').contents[19]

In [87]:
trait_key = str(temp).split('>')[2].rstrip('</span')
trait_val = str(temp).split('>')[3].rstrip('</li').strip()
print(trait_key, ':', trait_val)

Penalty : Disciplined


In [ ]:
<div class="player-traits">
<h5>Player Traits</h5>
<ul class="player-traits-list">
<li><span>Clutch</span> Yes</li>
<li><span>Covers Ball</span> Brace for All Hits</li>
<li><span>Fights for Extra Yards</span> Yes</li>
<li><span>Makes Aggressive Catches</span> Yes</li>
<li><span>Makes RAC Catches</span> Yes</li>
<li><span>Makes Possession Catches</span> Yes</li>
<li><span>Drops Open Passes</span> No</li>
<li><span>Makes Sideline Catches</span> Yes</li>
<li><span>High Motor</span> Yes</li>
<li><span>Penalty</span> Disciplined</li>
</ul>
</div>

In [114]:
ph = PlayerHandler()

wrs = ph.handle_players()

In [117]:
wr_stat_list = []
wr_trait_list = []

for wr in wrs:
    wr.summary()
    wr_stat_list.append(wr.get_stats_df())
    wr_trait_list.append(wr.get_traits_df())

wr_stats = pd.concat(wr_stat_list)
wr_traits = pd.concat(wr_trait_list)

for _ in wr_stats.columns:
    wr_stats[_] = wr_stats[_].astype(int)
    
all_data = pd.concat([wr_stats, wr_traits], axis=1)

all_data.to_csv('mut_wrs_90plus_nov17.csv')

calvin johnson  -  690K   -  1
andre reed  -  782K   -  1
julian edelman  -  605K   -  1
larry fitzgerald  -  660K   -  1
jarvis landry  -  625K   -  1
steve smith sr  -  350K   -  1
adam thielen  -  436K   -  1
odell beckham jr  -  802K   -  1
antonio brown  -  335K   -  1
deandre hopkins  -  321K   -  1
alshon jeffery  -  321K   -  1
herman moore  -  600K   -  1
steve smith sr  -  680K   -  1
keenan allen  -  195K   -  1
tim brown  -  145K   -  1
mike evans  -  626K   -  1
t y hilton  -  578K   -  1
julio jones  -  223K   -  1
randy moss  -  270K   -  1
jordy nelson  -  160K   -  1
steve smith sr  -  150K   -  1
golden tate iii  -  186K   -  1
michael thomas  -  240K   -  1
kelvin benjamin  -  142K   -  1
brandin cooks  -  227K   -  1


,ACC,AGI,AWR,BCV,BTK,CAR,CIT,CTH,DRR,ELU,...,Clutch,Covers Ball,Drops Open Passe,Fights for Extra Yard,High Motor,Makes Aggressive Catche,Makes Possession Catche,Makes RAC Catche,Makes Sideline Catche,Penalty
calvin johnson,92,93,89,87,85,80,92,90,93,85,...,Yes,Brace for All Hits,No,Yes,Yes,Yes,Yes,Yes,Yes,Disciplined
andre reed,91,93,93,88,83,76,92,93,90,87,...,Yes,Brace vs. Big Hitters,No,Yes,Yes,No,Yes,Yes,Yes,Norma
julian edelman,91,91,95,91,83,72,94,92,82,87,...,Yes,Brace for All Hits,No,Yes,Yes,No,Yes,Yes,Yes,Disciplined
larry fitzgerald,88,85,95,91,71,71,93,95,85,71,...,Yes,Brace for All Hits,No,Yes,Yes,Yes,Yes,Yes,Yes,Norma
jarvis landry,89,90,93,91,84,73,90,95,85,86,...,No,Brace for All Hits,No,Yes,Yes,Yes,No,Yes,Yes,Norma
steve smith sr,92,91,91,91,88,71,92,91,90,90,...,Yes,Brace for All Hits,No,Yes,Yes,Yes,No,Yes,Yes,Norma
adam thielen,91,91,92,87,78,68,89,93,88,77,...,No,Brace for All Hits,No,No,Yes,Yes,Yes,Yes,Yes,Norma
odell beckham jr,90,93,92,90,85,69,89,88,90,90,...,Yes,Brace for All Hits,No,Yes,No,Yes,No,Yes,Yes,Undisciplined
antonio brown,89,94,92,92,82,73,91,92,88,90,...,Yes,Brace for All Hits,No,Yes,Yes,Yes,Yes,Yes,Yes,Undisciplined
deandre hopkins,89,88,92,88,75,76,95,92,86,75,...,Yes,Brace for All Hits,No,Yes,Yes,Yes,Yes,Yes,Yes,Undisciplined


In [395]:
#temp = soup.find_all('div', class_='price-container')[1].contents[3]
soup.find_all('span', class_='item-price')[1].contents[0]


'104K '

In [410]:
ph.player_list[0].get_traits_df()

""
calvin johnson


In [419]:
for i in range(len(wrs)):
    print(i, ':', wrs[i].stat_dict.keys())
    print()

0 : dict_keys([])

1 : dict_keys([])

2 : dict_keys([])

3 : dict_keys([])

4 : dict_keys([])

5 : dict_keys([])

6 : dict_keys([])

7 : dict_keys([])

8 : dict_keys([])

9 : dict_keys([])

10 : dict_keys([])

11 : dict_keys([])

12 : dict_keys([])

13 : dict_keys([])

14 : dict_keys([])

15 : dict_keys([])

16 : dict_keys([])

17 : dict_keys([])

18 : dict_keys([])

19 : dict_keys([])

20 : dict_keys([])

21 : dict_keys([])

22 : dict_keys([])

23 : dict_keys([])

24 : dict_keys([])



In [ ]:
<span class="overall">90</span>
<span class="position">WR</span>

<div class="right-bar">
<ul class="chemistries">
<li class="tip" data-count="1" title="Cincinnati Bengals | Boosts all Bengals players."><img src="https://media-touchdown.cursecdn.com/avatars/thumbnails/184/658/24/24/avatar.png"/></li>
</ul>
</div>
<img class="program-icon flashbacks" src="https://media-touchdown.cursecdn.com/avatars/thumbnails/183/512/32/32/avatar.png"/>


<div class="name">
<span class="first-and-last-name">A.J. Green</span>

<div class="program-name">Flashbacks</div>

<span class="cap-hit">40</span>

<div class="name-stats">
<span class="first-name">A.J.</span>
<span class="last-name">Green</span>
<span class="team">Cincinnati Bengals</span>
<span class="height-weight">Ht: 6' 4" Wt: 210</span>

In [343]:
ovr = int(soup.find('span', class_='overall').string)
pos = soup.find('span', class_='position').string
program = soup.find('div', class_='program-name').string
firstname = soup.find('span', class_='first-name').string
lastname = soup.find('span', class_='last-name').string
team = soup.find('span', class_='team').string
htwt = soup.find('span', class_='height-weight').string
caphit = int(soup.find('span', class_='cap-hit').string)

In [287]:
[int(s) for s in str.split() if s.isdigit()]

[]

In [342]:
ovr_str

'90'

In [273]:
for t in valid_traits:
    print(len(t))

6
11
22
24
17
24
17
22
10
7


In [280]:
response = requests.get(p_list[0].full_link)
soup = BeautifulSoup(response.text, 'html.parser')
tag_list = []
stat_list = []
for sec_tag in soup.find_all('section', {'class':'player-details'}):
    for span_tag in sec_tag.find_all('ul', {'class':'player-details-stats'}):
        for i, li_tag in enumerate(sec_tag.find_all('li')):
            for j, stat in enumerate(li_tag.find_all('li')):
                print(i, '-', j, stat)
                stat_list.append(stat)
            tag_list.append(str(li_tag))

0 - 0 <li data-stat-abbreviation="SPD"><span class="stat stat-green-dark" data-stat-value="85">85</span> <span class="stat-abbreviation">SPD</span></li>
0 - 1 <li data-stat-abbreviation="ACC"><span class="stat stat-green-dark" data-stat-value="85">85</span> <span class="stat-abbreviation">ACC</span></li>
0 - 2 <li data-stat-abbreviation="AGI"><span class="stat stat-green-dark" data-stat-value="89">89</span> <span class="stat-abbreviation">AGI</span></li>
0 - 3 <li data-stat-abbreviation="STR"><span class="stat stat-orange-dark" data-stat-value="67">67</span> <span class="stat-abbreviation">STR</span></li>
0 - 4 <li data-stat-abbreviation="JMP"><span class="stat stat-green-light" data-stat-value="92">92</span> <span class="stat-abbreviation">JMP</span></li>
0 - 5 <li data-stat-abbreviation="AWR"><span class="stat stat-green-light" data-stat-value="91">91</span> <span class="stat-abbreviation">AWR</span></li>
7 - 0 <li data-stat-abbreviation="CTH"><span class="stat stat-green-dark" data-

203 - 1 <li>+1 ACC</li>
206 - 0 <li><img src="https://media-touchdown.cursecdn.com/avatars/thumbnails/184/658/20/20/avatar.png"/></li>


soup
<div class="price-container">
<span class="console-icon ps4"></span>
<span class="item-price">105K <span class="price-diff neg">-10.2%</span></span>

In [217]:
temp = str(soup.find_all('div', class_='price-container')[1]).split('\n')[2]
temp = temp.split('"item-price">')[1]
price = temp.split('<span class="price-diff neg">')[0].strip()
price_diff = temp.split('<span class="price-diff neg">')[1].rstrip('</span></span>')

'-10.2%'

In [116]:
#upgrade_number = 0
#new_link = link + f'/upgrades?from={upgrade_number}'
new_url = prefix + link + f'/upgrades?from={upgrade_number}'
new_url

'https://www.muthead.com/19/players/60416-a-j-green/upgrades'

In [233]:
pd.([ pd.DataFrame({wrs[0].name : wrs[0].stat_dict}).T, pd.DataFrame({wrs[1].name : wrs[1].stat_dict}).T], axis=1)

,ACC,AGI,AWR,BCV,BTK,CAR,CIT,CTH,DRR,ELU,...,RBK,RBP,RLS,SFA,SPC,SPD,SPM,SRR,STR,TRK
larry fitzgerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72.0,50.0,93.0,77.0,93.0,83.0,78.0,93.0,77.0,73.0
steve smith sr,90.0,88.0,88.0,89.0,86.0,69.0,90.0,90.0,89.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
x = pd.DataFrame({wrs[0].name : wrs[0].stat_dict}).T
y = pd.DataFrame({wrs[1].name : wrs[1].stat_dict}).T


,ACC,AGI,AWR,BCV,BTK,CAR,CIT,CTH,DRR,ELU,...,RBK,RBP,RLS,SFA,SPC,SPD,SPM,SRR,STR,TRK
steve smith sr,90,88,88,89,86,69,90,90,89,88,...,60,47,82,89,85,88,87,90,69,67
larry fitzgerald,88,85,95,91,71,71,93,95,85,71,...,72,50,93,77,93,83,78,93,77,73


In [172]:
for _ in player_df.columns:
    player_df[_] = player_df[_].astype(int)

In [175]:
player_df.to_csv('mut19_wrs_oct6.csv')

In [167]:
player_df.columns
cols = ['CIT', 'CTH', 'SPC', 'DRR', 'MRR', 'SRR', 'SPD', 'STR'] # 'RLS','SPM',

**RBs:**

In [ ]:
links2 = []

for i in range(1,2):
    rb_url = 'https://www.muthead.com/19/players?filter-ovr-min=88&filter-market=3&&filter-position=2'
    response = requests.get(rb_url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    for link in soup.findAll('a', attrs={'href': re.compile("19/players/")}):
        links2.append(link.get('href'))
        
rb_links = [s for s in links2 if 'prices' not in s]
rb_links = list(set(rb_links))

In [227]:
def get_rb_stats(soup, name):

    player_dict = {}

    for sec_tag in soup.find_all('section', {'class':'player-details'}):
    
        for span_tag in sec_tag.find_all('ul', {'class':'player-details-stats'}):
    
            for i, li_tag in enumerate(sec_tag.find_all('li')):
            
                #if i not in [0, 7, 15, 21]:
                    x = str(li_tag)
                    x1 = x.lstrip('<li data-stat-abbreviation="')
                    stat = x1[0:3]
                    rating = re.sub('[^0-9]','', str(li_tag))[0:2]
                
                    player_dict.update({stat : rating})
                        
    return pd.DataFrame([player_dict], index=[name])[['ACC', 'AGI', 'BCV', 'BTK', 'CAR',
       'CIT', 'CTH', 'DRR', 'ELU', 'JKM', 'JMP', 'MRR', 'SFA', 'SPC', 'SPD',
       'SPM', 'SRR', 'STR', 'TRK']]

In [229]:
prefix = 'https://www.muthead.com'

rb_list= []

for link in rb_links:
    new_url = prefix + link + 'upgrades'
    response = requests.get(new_url)
    player_name = ' '.join(link.split('-')[1:])
    soup = BeautifulSoup(response.text, 'html.parser')
    rb_list.append( get_rb_stats(soup, player_name) )
    time.sleep(2)
    
rb_df = pd.concat(rb_list)

In [230]:
rb_df.head()

,ACC,AGI,BCV,BTK,CAR,CIT,CTH,DRR,ELU,JKM,JMP,MRR,SFA,SPC,SPD,SPM,SRR,STR,TRK
christian mccaffrey,91,94,84,82,87,81,83,69,90,91,86,77,74,82,90,92,84,73,61
eddie george,88,84,87,88,88,64,62,52,76,77,73,55,88,50,85,79,64,84,87
barry sanders,90,90,90,88,86,67,68,48,89,89,76,61,75,62,89,89,71,65,69
devonta freeman,88,92,90,87,87,67,70,48,92,93,82,56,74,73,88,93,63,72,86
leonard fournette,89,86,90,87,90,54,66,37,78,85,67,45,88,52,89,87,53,82,93


In [232]:
for _ in rb_df.columns:
    rb_df[_] = rb_df[_].astype(int)

In [233]:
rb_df.to_csv('mut19_rbs_oct6.csv')

### Learning Plotly 

In [190]:
import plotly as py
py.tools.set_credentials_file(username='xxxxxx',api_key='xxxxxx')

In [215]:
import plotly.plotly as py
import plotly.graph_objs as go

**Plot with hover attributes:**

In [206]:
# cols = ['CIT', 'CTH', 'SPC', 'DRR', 'MRR', 'SRR', 'SPD', 'STR'] # 'RLS','SPM',

df = player_df

trace1 = go.Splom(dimensions=[dict(label='CIT',
                                 values=df['CIT']),
                            dict(label='CTH',
                                 values=df['CTH']),
                            dict(label='SPC',
                                 values=df['SPC']),
                            dict(label='SPD',
                                 values=df['SPD']),
                            dict(label='DRR',
                                 values=df['DRR']),
                            dict(label='MRR',
                                 values=df['MRR']),
                             dict(label='STR',
                                 values=df['STR'])],
                text=df.index.values,
                marker=dict(size=7,
                            showscale=False,
                            line=dict(width=0.5,
                                      color='rgb(230,230,230)'))
                )


axis = dict(showline=True,
          zeroline=False,
          gridcolor='#fff',
          ticklen=4)

layout = go.Layout(
    title='MUT19 Wide Receivers (88+ OVR)',
    dragmode='select',
    width=1000,
    height=1000,
    autosize=False,
    hovermode='closest',
    plot_bgcolor='rgba(240,240,240, 0.95)',
    xaxis1=dict(axis), xaxis2=dict(axis), xaxis3=dict(axis), xaxis4=dict(axis),
    xaxis5=dict(axis), xaxis6=dict(axis), xaxis7=dict(axis), xaxis8=dict(axis),
    yaxis1=dict(axis), yaxis2=dict(axis), yaxis3=dict(axis), yaxis4=dict(axis),
    yaxis5=dict(axis), yaxis6=dict(axis), yaxis7=dict(axis), yaxis8=dict(axis)
)
trace1['diagonal'].update(visible=False)
trace1['showupperhalf']=False
fig1 = dict(data=[trace1], layout=layout)
py.iplot(fig1)

In [234]:
# cols = ['CIT', 'CTH', 'SPC', 'DRR', 'MRR', 'SRR', 'SPD', 'STR'] # 'RLS','SPM',

df = rb_df

trace1 = go.Splom(dimensions=[dict(label='SPD',
                                 values=df['SPD']),
                            dict(label='SPM',
                                 values=df['SPM']),
                            dict(label='AGI',
                                 values=df['AGI']),
                            dict(label='ELU',
                                 values=df['ELU']),
                            dict(label='TRK',
                                 values=df['TRK']),
                            dict(label='BTK',
                                 values=df['BTK']),
                             dict(label='STR',
                                 values=df['STR'])],
                text=df.index.values,
                marker=dict(size=7,
                            showscale=False,
                            line=dict(width=0.5,
                                      color='rgb(230,230,230)'))
                )


axis = dict(showline=True,
          zeroline=False,
          gridcolor='#fff',
          ticklen=4)

layout = go.Layout(
    title='MUT19 Running Backs (88+ OVR)',
    dragmode='select',
    width=1000,
    height=1000,
    autosize=False,
    hovermode='closest',
    plot_bgcolor='rgba(240,240,240, 0.95)',
    xaxis1=dict(axis), xaxis2=dict(axis), xaxis3=dict(axis), xaxis4=dict(axis),
    xaxis5=dict(axis), xaxis6=dict(axis), xaxis7=dict(axis), xaxis8=dict(axis),
    yaxis1=dict(axis), yaxis2=dict(axis), yaxis3=dict(axis), yaxis4=dict(axis),
    yaxis5=dict(axis), yaxis6=dict(axis), yaxis7=dict(axis), yaxis8=dict(axis)
)
trace1['diagonal'].update(visible=False)
trace1['showupperhalf']=False
fig1 = dict(data=[trace1], layout=layout)
py.iplot(fig1)